In [1]:
import os
import arcpy
import numpy as np
import math
from datetime import datetime


dirPath = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\White-Goose-Light-Pollution\Data')
inVector = os.path.join(dirPath , 'Vector','points.shp')
biomeVector = os.path.join(dirPath,'Vector','tnc_terr_ecoregions.shp')
inRasterFolder = os.path.join(dirPath,'Images')
outRasterFolder = os.path.join(dirPath,'Images_Extent')

arcpy.env.workspace = inRasterFolder

## getting extent and adding 1000m
desc = arcpy.Describe(inVector)
extent = desc.extent
bufferDistance = 0.0001388  ##1000m in degree units
extentString = str(extent.XMin - bufferDistance) + ' ' + str(extent.YMin - bufferDistance) + ' ' +  \
                str(extent.XMax + bufferDistance) + ' ' + str(extent.YMax + bufferDistance) 
print(extentString)


###clipping each raster
inRasters = arcpy.ListRasters()
for raster in inRasters:
    rasterDesc =  arcpy.Describe(raster)
    rasterName = rasterDesc.name
    outRaster = os.path.join(outRasterFolder, rasterName)
    
    ##delete raster if exits
    if arcpy.Exists(outRaster):
        arcpy.Delete_management(outRaster)
    
    ## clip raster
    arcpy.Clip_management (raster, extentString, outRaster, \
                     maintain_clipping_extent = 'NO_MAINTAIN_EXTENT')

3.6740312 51.1431912 87.6671388 73.5753088


In [2]:
### Identifying breeding biomes
arcpy.MakeFeatureLayer_management(biomeVector, 'biome')
arcpy.SelectLayerByLocation_management('biome', 'INTERSECT', inVector)
arcpy.SelectLayerByAttribute_management('biome', 'SUBSET_SELECTION', " ECO_NAME LIKE '%Tundra%' ")
arcpy.CopyFeatures_management('biome', os.path.join(dirPath, 'tundra_breed.shp'))

tundraVector  = os.path.join(dirPath,'Vector','tundra_breed.shp')

In [3]:
def checkBreedingTime(date):
    for year in range(2006 , 2010):
        breedingStartTime = datetime(year, 5, 24)
        breedingEndTime = datetime(year, 6, 7)
        if date > breedingStartTime and date < breedingEndTime:
            isBreeding = True
            break
        else:
            isBreeding = False
    return isBreeding

In [4]:
def distanceKm(lat1, lon1, lat2, lon2):
    p = math.pi/180
    a = 0.5 - math.cos((lat2-lat1)*p)/2 + math.cos(lat1*p) * math.cos(lat2*p) * (1-math.cos((lon2-lon1)*p))/2
    distance = 2* 6371 * math.asin(math.sqrt(a))
    return distance

In [5]:
###classifying breeding season and adding time and date fields
listFields = [field.name for field in arcpy.ListFields(inVector)]
checkFields = {'isBreed':'SHORT', 'date':'DATE','speed':'DOUBLE', 'NTL_int': 'SHORT'}
for field, fieldType in checkFields.items():
    if field in listFields:
        arcpy.DeleteField_management(inVector, field)
    arcpy.AddField_management(inVector, field, fieldType)

    
fields = ['SHAPE@XY', 'tag_ident', 'timestamp', 'isBreed', 'date', 'speed']

with arcpy.da.UpdateCursor(inVector,fields) as cursor:
    index = 0
    for row in cursor:
        
        # Get current coordinates, time and goose id information 
        currentTime = datetime.strptime(row[2], '%Y-%m-%d %H:%M:%S')
        lon , lat = row[0]
        currentGroup = row[1]
        
        # Calculate values starting from the second feature for each group - white-fronted goose id -
        if index > 0:
            if currentGroup == previousGroup:
                distance = distanceKm(lat, lon, lat2, lon2)
                seconds = abs((currentTime - previousTime).total_seconds())
                speed = distance/seconds if seconds > 0 else 0
            else:
                distance = 0
        else:
            distance = 0
        
        ## updating fields
        row[3] = 0
        row[4] = currentTime.date()
        row[5] = distance
        
        # Set previos information by assigning current information - for next iteration
        previousTime = currentTime
        previousGroup = currentGroup
        lon2 = lon
        lat2 = lat
        
        index = index + 1
        cursor.updateRow(row)

In [6]:
## identify breeding points based on time and location
with arcpy.da.UpdateCursor(inVector,['SHAPE@XY','timestamp','isBreed']) as cursor:
    for row in cursor:
        x,y = row[0]
        time = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S')
        cursorShp = arcpy.SearchCursor(tundraVector)
        for rowid in cursorShp:
            poly = rowid.Shape
            if poly.contains(arcpy.Point(x,y)) and checkBreedingTime(time):
                row[2] = 1
        cursor.updateRow(row)

In [ ]:
arcpy.env.workspace = outRasterFolder
lightRasters = arcpy.ListRasters()[0:2] 

with arcpy.da.UpdateCursor(inVector,['SHAPE@XY','date','NTL_int']) as cursor:
    for year in range(2007,2009):
        for row in cursor:
            for raster in lightRasters:
                rasterDesc =  arcpy.Describe(raster)
                if row[1].year == year and str(year) in rasterDesc.name:
                    x, y = row[0]
                    coords = str(x) + " " + str(y)
                    row[2] = float(arcpy.GetCellValue_management(raster, coords)[0])
                    continue
            cursor.updateRow(row)

In [ ]:
inVectorArray = arcpy.da.FeatureClassToNumPyArray(inVector, ('isBreed','NTL_int'))
breedingCount = (inVectorArray['isBreed'] == 1).sum()
nonBreedingCount = (inVectorArray['isBreed'] == 0).sum()

if inVectorArray['isBreed'] == 1:
    meanLightBreed = inVectorArray['NTL_int'].mean()

print('Breeding points: ', breedingCount)
print('Non Breeding points: ', nonBreedingCount)
print('Breeding Light Mean: ', meanLightBreed)